<div class="usecase-title">Impact of Greenhouse Gas Emissions</div>

<div class="usecase-authors"><b>Authored by: </b> Chathumini Satharasinghe</div>

<div class="usecase-duration"><b>Duration:</b> 270 mins</div>

<div class="usecase-level-skill">
    <div class="usecase-level"><b>Level: </b>Intermediate</div>
    <div class="usecase-skill"><b>Pre-requisite Skills: </b>Python, Machine Learning</div>
</div>

<div class="usecase-section-header">Scenario</div>

As an environmental data analyst, I want to explore the relationship between greenhouse gas (GHG) emissions and microclimate conditions in Melbourne so that I can understand the impact of emissions on urban temperature and humidity trends over time and support sustainable urban planning.

<div class="usecase-section-header">What this use case will teach you</div>

This use case will teach you how to integrate environmental datasets from Melbourne’s open data portal, clean and process them, and analyze the connection between urban greenhouse gas emissions and climate variables such as temperature and humidity. You'll also learn how to visualize trends and correlations, and generate data-driven insights that can inform policy or urban design strategies.

At the end of this use case you will:
- Demonstrate skills in environmental data acquisition and integration.
- Apply data wrangling techniques on multi-source open datasets.
- Perform exploratory data analysis (EDA) to identify patterns and trends.
- Conduct correlation analysis between emissions and climate indicators.
- Create visualizations using Python to communicate findings effectively.
- Derive insights for sustainable development and climate adaptation

<div class="usecase-section-header"><b>Understanding the Impact of Greenhouse Gas Emissions on Melbourne’s Climate</b></div>

The rising levels of greenhouse gas (GHG) emissions are a major contributor to climate change, particularly in urban areas where industrial activity, transport, and energy consumption are concentrated. Understanding how these emissions affect local climate conditions, such as temperature and humidity, is critical for city planners and policymakers aiming to develop sustainable solutions.

This use case explores the relationship between GHG emissions and microclimate changes in Melbourne. It leverages two publicly available datasets from the City of Melbourne Open Data Platform:

1. **City of Melbourne Municipal Greenhouse Gas Emissions (2013–2020):**  
   This dataset provides annual emissions data (in tonnes of CO₂ equivalent), segmented by emission source (e.g., transport, energy, waste) and emission type (Council vs. Community).  
   👉 [Access Dataset](https://data.melbourne.vic.gov.au/explore/dataset/city-of-melbourne-municipal-greenhouse-gas-emissions-2013-2020/information/)

2. **Microclimate Sensor Readings:**  
   This dataset contains hourly environmental sensor data collected from various Melbourne locations, including temperature, humidity, solar radiation, and wind speed readings.  
   👉 [Access Dataset](https://data.melbourne.vic.gov.au/explore/dataset/microclimate-sensor-readings/information/)

By combining these datasets, this use case aims to identify trends and correlations between emissions and local climate variables, offering insights that support evidence-based environmental and infrastructure policy-making.



In [6]:
!pip install statsmodels

In [6]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime
import warnings
warnings.filterwarnings('ignore')
from scipy.stats import pearsonr
# import statsmodels.api as sm  # Now this should work
import requests
from io import StringIO

**Function to Request Datasets via API**

In [7]:
def request_data(dataset_id, format='csv', delimiter=';', api_key=None):
    base_url = 'https://data.melbourne.vic.gov.au/api/explore/v2.1/catalog/datasets/'
    url = f'{base_url}{dataset_id}/exports/{format}'

    params = {
        'select': '*',
        'limit': -1,
        'lang': 'en',
        'timezone': 'UTC'
    }

    headers = {}
    if api_key:
        headers['Authorization'] = f'Bearer {api_key}'

    try:
        response = requests.get(url, params=params, headers=headers, timeout=10)
        response.raise_for_status()

        if format == 'csv':
            content = response.content.decode('utf-8')
            df = pd.read_csv(StringIO(content), delimiter=delimiter)
            return df
        else:
            raise ValueError(f"Unsupported format: {format}")

    except requests.exceptions.RequestException as e:
        raise Exception(f'API request failed: {e}')

**Load Datasets**

1. Greenhouse Gas Emissions Dataset

In [10]:
emissions_df = request_data("city-of-melbourne-municipal-greenhouse-gas-emissions-2013-2020")


2. Microclimate Sensor Readings Dataset
   

In [12]:
climate_df = request_data("microclimate-sensor-readings", delimiter=',')

In [15]:
emissions_df = request_data("city-of-melbourne-municipal-greenhouse-gas-emissions-2013-2020")
emissions_df.head()

year  population  area_km2  gdp_m  residential_buildings_scope_1  \
0  2019      178955  37.7 km2  70221                          37913   
1  2018      169961   37.7km2  66472                          37520   
2  2017      152992   37.7km2  69780                          35733   
3  2016      148000        38  68859                          35547   
4  2015      139000        38  68859                          35637   

   residential_buildings_scope_2  residential_buildings_scope_3  \
0                         276303                          27089   
1                         256112                          23936   
2                         262254                          24283   
3                         257854                          23656   
4                         264951                          30753   

   commercial_and_institutional_buildings_and_facilities_scope_1  \
0                                             170602               
1                                             165523               
2                                             147148               
3                                             131662               
4                                             131662               

   commercial_and_institutional_buildings_and_facilities_scope_2  \
0                                            2714112               
1                                            2876672               
2                                            2938098               
3                                            2608552               
4                                            2680347               

   commercial_and_institutional_buildings_and_facilities_scope_3  ...  \
0                                             266089              ...   
1                                             268848              ...   
2                                             272046              ...   
3                                             239317              ...   
4                                             283790              ...   

   wastewater_generated_in_the_city_scope_2  \
0                                      7847   
1                                      7580   
2                                      6709   
3                                      3042   
4                                      3042   

   wastewater_generated_in_the_city_scope_3  \
0                                     False   
1                                     False   
2                                     False   
3                                     False   
4                                     False   

   emissions_from_industrial_processes_occurring_in_the_city_boundary_scope_1  \
0                                                 NE                            
1                                                 NE                            
2                                                 NE                            
3                                                 NE                            
4                                                 NE                            

  emissions_from_product_use_occurring_within_the_city_boundary_scope_2  \
0                                                 NE                      
1                                                 NE                      
2                                                 NE                      
3                                                 NE                      
4                                                 NE                      

  emissions_from_livestock_scope_3 emissions_from_land_scope_1  \
0                               NE                          NE   
1                               NE                          NE   
2                               NE                          NE   
3                               NE                          NE   
4                               NE                          NE   

  emissions_fro

In [14]:
microclimate_sensors_data = request_data("microclimate-sensor-readings")
microclimate_sensors_data.head()

local_time       id  site_id sensor_id    value  \
0  2023-11-29T22:15:02+00:00  8110553     1011        5b    74.38   
1  2023-11-29T22:15:02+00:00  8110558     1011        5c  1010.80   
2  2023-11-29T22:15:03+00:00  8110562     1011         6     0.00   
3  2023-11-29T22:15:04+00:00  8110606     1012         6     0.00   
4  2023-11-29T22:15:11+00:00  8110489     1009        0a     5.30   

           type   units gatewayhub_id site_status  
0        TPH.RH       %       arc1055           C  
1  TPH.PRESSURE     hPa       arc1055           C  
2            WS    km/h       arc1055           C  
3            WS    km/h       arc1048           C  
4         PM2.5  ug/m^3       arc1050           C